In [2]:
import socket
import threading
import queue
from tkinter import *
from tkinter import simpledialog
from tkinter.scrolledtext import ScrolledText #
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad

'''DES Encryption'''
des_key = b'password'
cipher = DES.new(des_key, DES.MODE_ECB)

'''Connect To Server'''
# Create a socket object for the client
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)   
# Connect to the server
client.connect(('localhost', 43210)) 

'''Initialise tkinter root'''
root = Tk()
root.withdraw()

'''Nickname'''
nickname = simpledialog.askstring("Nickname", "Please choose a nickname", parent=root)  # Ask the user for a nickname
#Close the Tkinter window (this wasn't closing and causing Python to hang)
root.destroy()

messages_queue = queue.Queue()

def update_gui_from_queue():
    while not messages_queue.empty():
        message = messages_queue.get()
        if message:
            text_area.config(state=NORMAL)
            text_area.insert(END, message + "\n")
            text_area.config(state=DISABLED)
            text_area.yview(END)
    root.after(100, update_gui_from_queue)

'''Building GUI with tkinter - the chat window'''
def run_gui():
    
    global root, text_area, message_entry
    
    root = Tk()
    root.title("Secure Chat App - " + nickname)
    
    #Text area showing messages
    frame = Frame(root)
    scrollbar = Scrollbar(frame)
    text_area = ScrolledText(frame, height = 20, width = 50, yscrollcommand = scrollbar.set, state = DISABLED)
    scrollbar.pack(side = RIGHT, fill = Y)
    text_area.pack(side = LEFT, fill= BOTH)
    frame.pack()
    
    
    #Message box
    message_entry = Entry(root, width = 50)
    message_entry.bind("<Return>", send_msg) #allow the enter key to send message
    message_entry.pack()
    
    #Send button
    send_button = Button(root, text="Send", command = send_msg)
    send_button.pack()
    

    root.protocol("WM_DELETE_WINDOW", on_closing)  # Handle window closing event
    root.after(100, update_gui_from_queue)
    root.mainloop()




# Listening to Server and Sending Nickname
def receive():
    """Handle messages from the server."""
    while True:
        try:
            message = client.recv(1024).decode('utf-8')  # Try to receive message from the server
            if message == 'NICK':
                client.send(nickname.encode('utf-8'))  # If asked for nickname, send it
            else:
                #If message received, insert into text area (update_gui_from_queue() above)
                messages_queue.put(message)
        except Exception as e:
            # Handle any errors
            print(f"An error occurred: {e}")
            print("Oh no! An error has occurred!")
            client.close()
            break

            
'''    Originally used in CLI version, not used in GUI. Keeping for later / just in case.       
def write():

    while True:
        message = f'{nickname}: {input("")}'
        client.send(message.encode('utf-8'))
''' 

'''Send meesages'''
def send_msg(event=None):  # event is passed by binders.
    message = message_entry.get()
    if message:  # If there's a message in the entry widget
        padded_message = pad(message.encode(), DES.block_size)  # Pad message to be multiple of 8 bytes
        encrypted_message = cipher.encrypt(padded_message)
        #final_message = f'{nickname}: {message}'  # keep this - added the above for encrypted messages
        try:
            client.send(encrypted_message)  # Send encrypted message
            #client.send(final_message.encode('utf-8')) # keep this - added the above for encrypted messages
        #adding in better error messaging for debug    
        except Exception as e:
            print(f"Failed to send message: {e}")
            
        message_entry.delete(0, END)  # Clear the input field after sending a message


    
''' function to handle window closing. '''    
def on_closing(event=None):
    try:
        client.send(f"{nickname} has left the chat.".encode('utf-8'))
        client.close()  # Attempt to close the connection gracefully
    except Exception as e:
        print(f"Error closing the client socket: {e}")
        
    root.destroy()  # Destroy the Tkinter window

# Starting the GUI and the receive thread
if __name__ == "__main__":
    receive_thread = threading.Thread(target=receive)
    receive_thread.start()
    run_gui()    
    
    
''' To be removed
#Starting threads
#Start the receiving thread
receive_thread = threading.Thread(target = receive)
receive_thread.start()

#Start the GUI thread
write_thread = threading.Thread(target = run_gui)
write_thread.start()
'''


bono joined the chat!bono joined the chat!
Connected to the server!

bono: Hijono left the chat!

bono: Hello
bono: I am here
